# Set List Table

## Introduction

The purpose of this notebook is to process and upload keyword data from MTGJSON into the postgresql database mtg_db. This is done through the following steps:
- Download the json file from MTGJSON's file server
- Check the version and date of the json file
- Pre-process the dictionary and convert it into a dataframe
- Push the keywords dataframe to the database "raw_data" schema

## Schemas

### Input table - Set List

| Column            | Renamed         | Datatype   | Description                                                                |
| ---               | ---             | ---        | ---                                                                        |
| code              | SET_CODE        | STRING     | The set code                                                               |
| name              | SET_NAME        | STRING     | Name of the set                                                            |
| baseSetSize       | BASE_SET_SIZE   | INTEGER    | The number of cards in the base set without promos or supplements          |
| cardsphereSetId   | CS_SET_ID       | FLOAT      | ID for set in Cardsphere                                                   |
| mcmId             | CM_ID           | FLOAT      | Card Market set ID                                                         |
| mcmIdExtras       | CM_ID_ADD       | FLOAT      | If the set is split into two sets this is the additional Card Market ID    |
| mcmName           | CM_NAME         | STRING     | Name of the set on Card Market                                             |
| isFoilOnly        | FOIL_FLAG       | BOOLEAN    | Flag whether the set is only available as foils                            |
| isForeignOnly     | FOREIGN_FLAG    | BOOLEAN    | Flag whether the set is only available outside the US                      |
| keyruneCode       | KEYRUNE_CODE    | STRING     | ID for the keyrune database of set icons                                   |
| languages         | LANGUAGES       | LIST       | List of languages the set was printed in                                   |
| mtgoCode          | MTGO_SET_CODE   | STRING     | Set code on Magic The Gathering Online                                     |
| isNonFoilOnly     | NON_FOIL_FLAG   | BOOLEAN    | Flag whether the set is only available as non-foils                        |
| isOnlineOnly      | ONLINE_FLAG     | BOOLEAN    | Flag whether the set is only available in online formats                   |
| isPartialPreview  | PREVIEW_FLAG    | BOOLEAN    | Flag whether the set is still in preview and not complete                  |
| sealedProduct     | PRODUCT_INFO    | LIST       | Information about the purchasable sealed product                           |
| releaseDate       | RELEASE_DATE    | STRING     | Date the set was release, in format YYYY-MM-DD                             |
| block             | SET_BLOCK_NAME  | STRING     | Block the set is in, e.g. Kaladesh                                         |
| decks             | SET_DECKS       | LIST       | All decks associated with the set                                          |
| parentCode        | SET_PARENT_CODE | STRING     | Code of the parent set for set variations, e.g. promotions, guild kits etc |
| tokenSetCode      | SET_TOKEN_CODE  | STRING     | Code for the set's tokens                                                  |
| type              | SET_TYPE        | STRING     | The type of set, e.g. alchemy, commander, funny                            |
| tcgplayerGroupId  | TCGPG_ID        | INTEGER    | ID for the set on TCGplayer                                                |
| totalSetSize      | TOTAL_SET_SIZE  | INTEGER    | The number opf cards in the set with promos and supplements                |
| translations      | TRANSLATIONS    | DICTIONARY | The translated name of the set                                             |

### Main Table - Sets

Schema for df__sets

| Column          | Datatype   | Description                                                                |
| ---             | ---        | ---                                                                        |
| SET_CODE        | STRING     | The set code                                                               |
| SET_NAME        | STRING     | Name of the set                                                            |
| RELEASE_DATE    | STRING     | Date the set was release, in format YYYY-MM-DD                             |
| SET_TYPE        | STRING     | The type of set, e.g. alchemy, commander, funny                            |
| SET_BLOCK_NAME  | STRING     | Block the set is in, e.g. Kaladesh                                         |
| SET_PARENT_CODE | STRING     | Code of the parent set for set variations, e.g. promotions, guild kits etc |
| SET_TOKEN_CODE  | STRING     | Code for the set's tokens                                                  |
| BASE_SET_SIZE   | INTEGER    | The number of cards in the base set without promos or supplements          |
| TOTAL_SET_SIZE  | INTEGER    | The number opf cards in the set with promos and supplements                |
| DECK_COUNT      | INTEGER    | The number of decks released with the set                                  |
| FOIL_FLAG       | BOOLEAN    | Flag whether the set is only available as foils                            |
| NON_FOIL_FLAG   | BOOLEAN    | Flag whether the set is only available as non-foils                        |
| FOREIGN_FLAG    | BOOLEAN    | Flag whether the set is only available outside the US                      |
| ONLINE_FLAG     | BOOLEAN    | Flag whether the set is only available in online formats                   |
| PREVIEW_FLAG    | BOOLEAN    | Flag whether the set is still in preview and not complete                  |
| CM_ID           | INTEGER    | Card Market set ID                                                         |
| CM_ID_ADD       | INTEGER    | If the set is split into two sets this is the additional Card Market ID    |
| CM_NAME         | STRING     | Name of the set on Card Market                                             |
| CS_SET_ID       | INTEGER    | ID for set in Cardsphere                                                   |
| KEYRUNE_CODE    | STRING     | ID for the keyrune database of set icons                                   |
| MTGO_SET_CODE   | STRING     | Set code on Magic The Gathering Online                                     |
| TCGPG_ID        | INTEGER    | ID for the set on TCGplayer                                                |

### Set Name Translations table

Schema for df__translations

| Column               | Datatype | Description                                         |
| ---                  | ---      | ---                                                 |
| SET_CODE             | STRING   | Set code ID                                         |
| SET_NAME             | STRING   | Name of the set                                     |
| BRAZILIAN_PORTUGUESE | STRING   | Portuguese set name translation if exists           |
| FRENCH               | STRING   | French set name translation if exists               |
| GERMAN               | STRING   | German set name translation if exists               |
| ITALIAN              | STRING   | Italian set name translation if exists              |
| JAPANESE             | STRING   | Japanese set name translation if exists             |
| KOREAN               | STRING   | Korean set name translation if exists               |
| RUSSIAN              | STRING   | Russian set name translation if exists              |
| SIMPLIFIED_CHINESE   | STRING   | Simplified Chinese set name translation if exists   |
| SPANISH              | STRING   | Spanish set name translation if exists              |
| TRADITIONAL_CHINESE  | STRING   | Traditional Chinese set name translation if exists  |


### Card Languages table

Schema for df__languages

| Column              | Datatype | Description                                                     |
| ---                 | ---      | ---                                                             |
| SET_CODE            | STRING   | Set code ID                                                     |
| SET_NAME            | STRING   | Name of the set                                                 |
| ANCIENT_GREEK       | BOOLEAN  | True/False whether the set is translated to Ancient Greek       |
| ARABIC              | BOOLEAN  | True/False whether the set is translated to Arabic              |
| CHINESE_SIMPLIFIED  | BOOLEAN  | True/False whether the set is translated to simplified Chinese  |
| CHINESE_TRADITIONAL | BOOLEAN  | True/False whether the set is translated to traditional Chinese |
| ENGLISH             | BOOLEAN  | True/False whether the set is translated to English             |
| FRENCH              | BOOLEAN  | True/False whether the set is translated to French              |
| GERMAN              | BOOLEAN  | True/False whether the set is translated to German              |
| HEBREW              | BOOLEAN  | True/False whether the set is translated to Hebrew              |
| ITALIAN             | BOOLEAN  | True/False whether the set is translated to Italian             |
| JAPANESE            | BOOLEAN  | True/False whether the set is translated to Japanese            |
| KOREAN              | BOOLEAN  | True/False whether the set is translated to Korean              |
| LATIN               | BOOLEAN  | True/False whether the set is translated to Latin               |
| PHYREXIAN           | BOOLEAN  | True/False whether the set is translated to Phyrexian           |
| BRAZILIAN_PORTUGESE | BOOLEAN  | True/False whether the set is translated to Portugese           |
| RUSSIAN             | BOOLEAN  | True/False whether the set is translated to Russian             |
| SANSKRIT            | BOOLEAN  | True/False whether the set is translated to Sanskrit            |
| SPANISH             | BOOLEAN  | True/False whether the set is translated to Spanish             |

### Set Decks table

### Set List Schema - Product Info table

## Python Libraries

In [1]:
import json
import requests
import lzma
from   tqdm                           import tqdm
import numpy                          as     np
import pandas                         as     pd
from   sqlalchemy                     import create_engine, Table, Column, MetaData, Text, Date, text
from   sqlalchemy.dialects.postgresql import insert
from   IPython.display                import HTML

In [2]:
# Show all columns instead of truncating with "..."
pd.set_option("display.max_columns", None)

# (Optional) also show all rows
pd.set_option("display.max_rows", None)

# (Optional) widen the display area so columns don’t wrap badly
pd.set_option("display.width", None)

## Functions

In [3]:
# Function for showing the hierarchy of a dictionary or the schema of a single level
def print_dict_structure(data, max_depth=None, _indent=0):

    """
    Recursively prints the hierarchical structure of a dictionary or list,
    including the length of each element where applicable.
    
    If max_depth=1, returns a DataFrame with columns: KEY_NAME, DATA_TYPE, LENGTH.
    
    Args:
        data: The dictionary or list to explore.
        max_depth: Limit how deep to traverse (None for full depth).
    
    Returns:
        pd.DataFrame if max_depth=1, otherwise None (prints output).
    """
    
    # Check if we are at the top level and max_depth=1 to return DataFrame instead of printing
    if max_depth == 1 and _indent == 0:
        # Initialize list to collect rows for the DataFrame
        rows = []

        # If data is a dictionary, iterate over its keys and values
        if isinstance(data, dict):
            for key, value in data.items():
                # Determine length if possible, otherwise set to 0
                length = len(value) if hasattr(value, "__len__") and not isinstance(value, (str, bytes)) else 0
                # Append a tuple of key name, data type, and length to rows
                rows.append((key, type(value).__name__, length))

        # If data is a list, take the first element (assumed dict) and do the same
        elif isinstance(data, list) and data:
            for key, value in data[0].items():
                # Determine length if possible, otherwise set to 0
                length = len(value) if hasattr(value, "__len__") and not isinstance(value, (str, bytes)) else 0
                # Append a tuple of key name, data type, and length to rows
                rows.append((key, type(value).__name__, length))

        # Convert collected rows into a DataFrame with specific column names
        return pd.DataFrame(rows, columns=["KEY_NAME", "DATA_TYPE", "LENGTH"])
    
    # Create a prefix for indentation when printing nested structures
    prefix = "  " * _indent

    # If data is a dictionary, iterate recursively
    if isinstance(data, dict):
        for key, value in data.items():
            # Prepare a string showing type and length for printing
            length_info = f", len={len(value)}" if hasattr(value, "__len__") and not isinstance(value, (str, bytes)) else ""
            # Print the key name, type, and length with proper indentation
            print(f"{prefix}{key} ({type(value).__name__}{length_info})")
            # Recurse into value if max_depth is not reached
            if max_depth is None or _indent < max_depth - 1:
                print_dict_structure(value, max_depth, _indent + 1)

    # If data is a list, recurse into the first element (assuming homogeneous elements)
    elif isinstance(data, list):
        if data and (max_depth is None or _indent < max_depth - 1):
            print_dict_structure(data[0], max_depth, _indent + 1)

    # For non-dict and non-list elements, print their type and length
    else:
        # Prepare a string showing type and length for printing
        length_info = f", len={len(data)}" if hasattr(data, "__len__") and not isinstance(data, (str, bytes)) else ""
        # Print the type and length with proper indentation
        print(f"{prefix}{type(data).__name__}{length_info}")

In [4]:
# Function for showing the data and version of the MTGJSON data
def data_recency_check(data, json_type):

    """
    Extract and display the version and date metadata from an MTGJSON dataset,
    and return this information as a DataFrame along with the JSON type.

    Parameters
    ----------
    data : dict
        MTGJSON data loaded from a JSON file, expected to contain a 'meta' key
        with 'date' and 'version' fields.

    json_type : str
        A string indicating the type or name of the JSON dataset being processed.
        This will be included in the output DataFrame.

    Returns
    -------
    pd.DataFrame
        A DataFrame with a single row and columns:
        - 'json_type': The provided JSON dataset type/name.
        - 'latest_date': The date the MTGJSON data was last updated.
        - 'latest_version': The MTGJSON model version.
    """

    # Create a DataFrame for the output
    df = pd.DataFrame({'json_type'      : [json_type]
                      ,'latest_date'    : [data['meta']['date']]
                      ,'latest_version' : [data['meta']['version']]})

    # Returning the values directly
    return(df)

In [5]:
# Function for uploading the recency check

def recency_check_upload(schema_name, table_name, dataframe):
    
    """
    Uploads recency check data from a Pandas DataFrame into a PostgreSQL table 
    with upsert (insert or update) logic.

    Each row from the DataFrame is inserted into the target table. If a row with the 
    same `json_type` (primary key) already exists, the corresponding `latest_date` 
    and `latest_version` values are updated instead.

    Parameters
    ----------
    schema_name : str
        Name of the PostgreSQL schema where the table resides.
    table_name : str
        Name of the PostgreSQL table to update or insert into.
    dataframe : pandas.DataFrame
        DataFrame containing the recency check data with columns:
        - 'json_type' (str): Identifier for the JSON file type.
        - 'latest_date' (datetime.date): Date of the latest file.
        - 'latest_version' (str): Version string of the latest file.

    Notes
    -----
    - Requires a global SQLAlchemy `engine` object to be defined.
    - Uses PostgreSQL's ON CONFLICT clause for upsert behavior.
    """

    # Create a MetaData object
    metadata = MetaData(schema=schema_name)
    
    # Define the Table object matching your PostgreSQL table
    json_recency_table = Table(table_name
                              ,metadata
                              ,Column('json_type' ,Text ,primary_key = True)
                              ,Column('latest_date' ,Date)
                              ,Column('latest_version' ,Text))
    
    # Upsert each row from your DataFrame
    with engine.begin() as conn:
        
        # Iterate through rows of the DataFrame
        for _, row in dataframe.iterrows():
            
            # Create an insert statement for the current row
            stmt = insert(json_recency_table).values(json_type      = row['json_type']
                                                    ,latest_date    = row['latest_date']
                                                    ,latest_version = row['latest_version'])
            
            # Add upsert logic to update on conflict
            stmt = stmt.on_conflict_do_update(index_elements = ['json_type']
                                             ,set_           = {'latest_date'    : row['latest_date']
                                                               ,'latest_version' : row['latest_version']})
            
            # Execute the statement
            conn.execute(stmt)

In [6]:
# Function for scrollable dataframe outputs
def scrollable_df(df, height=400):
    """
    Render a pandas DataFrame as a scrollable HTML table in Jupyter Notebook.

    This function wraps a DataFrame in a styled <div> block with a maximum height, 
    enabling vertical scrolling when the table is too large to display fully. 
    It is especially useful for exploring wide or long tables in Jupyter cells 
    without overwhelming the notebook display.

    Parameters
    ----------
    df : pandas.DataFrame
        The DataFrame to render.
    height : int, optional (default=400)
        The maximum height of the rendered table in pixels before scrolling is enabled.

    Returns
    -------
    IPython.display.HTML
        An HTML object containing the scrollable DataFrame.
    """
    # Wrap the DataFrame in an HTML <div> with restricted height
    # Use inline CSS: max-height sets the scroll area, overflow:auto enables scrolling
    return HTML(f'<div style="max-height:{height}px; overflow:auto;">{df.to_html()}</div>')

## Input

### Database Connection

In [7]:
## Setting up credentials for accessing postgresql "mtg_db" database

# Credentials for setting up connection to postgresql
user     = "postgres"
password = "as:123bpostgresql"
host     = "localhost"
port     = "5432"
database = "mtg_db"

# Engine connection to postgresql
engine = create_engine(f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}")

In [8]:
## Creating the empty data_recency table if not exists
query = """
        CREATE TABLE IF NOT EXISTS raw_data.data_recency (
         json_type      TEXT PRIMARY KEY
        ,latest_date    DATE
        ,latest_version TEXT);
        """
with engine.begin() as conn:
    conn.execute(text(query))

### Input Data

In [9]:
# URL for MTGJSON (example: Keywords.xz)
url = "https://mtgjson.com/api/v5/SetList.json.xz"

# Download the compressed file
response = requests.get(url)
response.raise_for_status()

# Prepare to track total size and read in chunks
total_size = int(response.headers.get('content-length', 0))  # total bytes, may be None
chunk_size = 1024 * 1024  # 1 MB per chunk
compressed_data = bytearray()  # store the downloaded bytes

# Iterate over response chunks, updating progress bar
with tqdm(total=total_size, unit='B', unit_scale=True, desc="Downloading") as pbar:
    for chunk in response.iter_content(chunk_size=chunk_size):
        if chunk:  # filter out keep-alive chunks
            compressed_data.extend(chunk)
            pbar.update(len(chunk))

# Decompress the .xz file
decompressed_bytes = lzma.decompress(compressed_data)

# Parse JSON into a dictionary
dict__set_list = json.loads(decompressed_bytes)

Downloading: 100%|██████████| 1.46M/1.46M [00:00<00:00, 1.60GB/s]


### Column Lists & Dictionaries

In [151]:
# Dictionary for renaming the original set list dataframe
columns__rename_set_list = {'baseSetSize'                      : 'BASE_SET_SIZE'
                           ,'code'                             : 'SET_CODE'
                           ,'isFoilOnly'                       : 'FOIL_FLAG'
                           ,'isOnlineOnly'                     : 'ONLINE_FLAG'
                           ,'keyruneCode'                      : 'KEYRUNE_CODE'
                           ,'languages'                        : 'LANGUAGES'
                           ,'name'                             : 'SET_NAME'
                           ,'releaseDate'                      : 'RELEASE_DATE'
                           ,'sealedProduct'                    : 'PRODUCT_INFO'
                           ,'tcgplayerGroupId'                 : 'TCGPG_ID'
                           ,'totalSetSize'                     : 'TOTAL_SET_SIZE'
                           ,'type'                             : 'SET_TYPE'
                           ,'block'                            : 'SET_BLOCK_NAME'
                           ,'isNonFoilOnly'                    : 'NON_FOIL_FLAG'
                           ,'parentCode'                       : 'SET_PARENT_CODE'
                           ,'mcmId'                            : 'CM_ID'
                           ,'mcmName'                          : 'CM_NAME'
                           ,'tokenSetCode'                     : 'SET_TOKEN_CODE'
                           ,'translations.Chinese Simplified'  : 'TRANSLATION_SIMPLIFIED_CHINESE'
                           ,'translations.Chinese Traditional' : 'TRANSLATION_TRADITIONAL_CHINESE'
                           ,'translations.French'              : 'TRANSLATION_FRENCH'
                           ,'translations.German'              : 'TRANSLATION_GERMAN'
                           ,'translations.Italian'             : 'TRANSLATION_ITALIAN'
                           ,'translations.Japanese'            : 'TRANSLATION_JAPANESE'
                           ,'translations.Korean'              : 'TRANSLATION_KOREAN'
                           ,'translations.Portuguese (Brazil)' : 'TRANSLATION_BRAZILIAN_PORTUGESE'
                           ,'translations.Russian'             : 'TRANSLATION_RUSSIAN'
                           ,'translations.Spanish'             : 'TRANSLATION_SPANISH'
                           ,'cardsphereSetId'                  : 'CS_SET_ID'
                           ,'decks'                            : 'SET_DECKS'
                           ,'mcmIdExtras'                      : 'CM_ID_ADD'
                           ,'mtgoCode'                         : 'MTGO_SET_CODE'
                           ,'isPartialPreview'                 : 'PREVIEW_FLAG'
                           ,'isForeignOnly'                    : 'FOREIGN_FLAG'}

In [147]:
# Columns for the main set table
columns__sets = ['SET_CODE'
                ,'SET_NAME'
                ,'RELEASE_DATE'
                ,'SET_TYPE'
                ,'SET_BLOCK_NAME'
                ,'SET_PARENT_CODE'
                ,'SET_TOKEN_CODE'
                ,'BASE_SET_SIZE'
                ,'TOTAL_SET_SIZE'
                ,'DECK_COUNT'
                ,'FOIL_FLAG'
                ,'NON_FOIL_FLAG'
                ,'FOREIGN_FLAG'
                ,'ONLINE_FLAG'
                ,'PREVIEW_FLAG'
                ,'CM_ID'
                ,'CM_ID_ADD'
                ,'CM_NAME'
                ,'CS_SET_ID'
                ,'KEYRUNE_CODE'
                ,'MTGO_SET_CODE'
                ,'TCGPG_ID']

In [152]:
# Dictionary for renaming the deck table columns
columns__rename_set_decks = {'code'               : 'SET_CODE'
                            ,'commander'          : 'COMMANDER'
                            ,'displayCommander'   : 'DISPLAY_COMMANDER'
                            ,'mainBoard'          : 'DECK_CARDS'
                            ,'name'               : 'DECK_NAME'
                            ,'planes'             : 'PLANES'
                            ,'releaseDate'        : 'RELEASE_DATE'
                            ,'schemes'            : 'SCHEMES'
                            ,'sealedProductUuids' : 'SEALED_PRODUCT_IDS'
                            ,'sideBoard'          : 'SIDE_BOARD_CARDS'
                            ,'type'               : 'DECK_TYPE'}

In [154]:
# Columns included in the master table for set decks
columns__set_deck_info = ['SET_CODE'
                         ,'SET_NAME'
                         ,'DECK_NAME'
                         ,'RELEASE_DATE'
                         ,'DECK_TYPE'
                         ,'COMMANDER']

## Pre-processing

In [12]:
# Checking the latest version of the input data
df__data_recency = data_recency_check(dict__set_list, 'set list')

In [ ]:
## Creating the main dataframe
# Converting the dictionary to a flattened dataframe
df__set_list = pd.json_normalize(dict__set_list['data'])

# Renaming the columns
df__set_list = df__set_list.rename(columns = columns__rename_set_list)

# Sorting the set list by release date date and set code
df__set_list = df__set_list.sort_values(by = ['RELEASE_DATE', 'SET_CODE']).reset_index(drop = True)

# Reordering the columns alphabetically with the set name and code first
first_cols = ["SET_CODE", "SET_NAME"]
other_cols = sorted([c for c in df__set_list.columns if c not in first_cols])
df__set_list = df__set_list[first_cols + other_cols]

# Counting the number of decks per set
df__set_list['DECK_COUNT'] = df__set_list['SET_DECKS'].apply(lambda x: len(x) if isinstance(x, list) else 0)

## Main Code

### Set Table

In [148]:
# Making a copy of the input dataframe
df__sets = df__set_list[columns__sets].copy()

# Converting the flag columns to booleans
for col in ['NON_FOIL_FLAG', 'PREVIEW_FLAG', 'FOREIGN_FLAG']:
    df__sets[col] = df__sets[col].where(df__sets[col].notna(), False).astype(bool)

# Converting ID columns to integers
for col in ['CM_ID', 'CM_ID_ADD', 'CS_SET_ID', 'TCGPG_ID']:
    df__sets[col] = df__sets[col].astype('Int64')

### Set Name Translations

In [82]:
## Extracting the set name translations into a separate dataframe

# Creating new dataframe for the set name translations
columns__translations = [column for column in df__set_list.filter(like="TRANSLATION").columns]
df__translations      = df__set_list[['SET_CODE'] + ['SET_NAME'] + columns__translations].copy()

# Renaming the columns for the translation columns
df__translations.columns = df__translations.columns.str.replace("TRANSLATION_", "", regex=False)

### Set Languages

In [83]:
## Extracting the set language releases into a separate dataframe

# Making a copy of the columns into a new dataframe
df__languages = df__set_list[['SET_CODE','SET_NAME','RELEASE_DATE','LANGUAGES']].copy()

# Converting the languages column into a dataframe
df__languages = df__languages.explode('LANGUAGES')
df__languages = (df__languages.assign(value=True).pivot_table(index      = ['SET_CODE','SET_NAME','RELEASE_DATE']
                                                             ,columns    = 'LANGUAGES'
                                                             ,values     = 'value'
                                                             ,fill_value = False).astype(bool).reset_index())

# Fixing the column names
df__languages.columns = df__languages.columns.str.upper()
df__languages.columns = df__languages.columns.str.replace(' ','_')
df__languages = df__languages.rename(columns = {'PORTUGUESE_(BRAZIL)' : 'BRAZILIAN_PORTUGUESE'})
df__languages.columns.name = None

# Reordering the dataframe by release date
df__languages = df__languages.sort_values(by = 'RELEASE_DATE').drop(columns = ['RELEASE_DATE']).reset_index(drop = True)


### Set Decks

In [159]:
# Copying the ID and deck data from the input set table
df__set_decks = df__set_list[['SET_CODE','SET_NAME','SET_DECKS']].copy()

# Reordering the columns
df__set_decks = df__set_decks[['SET_CODE'
                              ,'SET_NAME'
                              ,'SET_DECKS']]

# Replacing the NaN rows in the deck column with empty lists so json_normalize works
df__set_decks['SET_DECKS'] = df__set_decks['SET_DECKS'].apply(lambda x: x if isinstance(x, list) else [])

# Exploding the deck lists into individual rows of dictionaries
df__set_decks = df__set_decks.explode('SET_DECKS', ignore_index=True)

# Converting the deck dictionaries into dataframes and joining back onto the table
df__set_decks = df__set_decks.merge(pd.json_normalize(df__set_decks['SET_DECKS'])
                                   ,left_on  = 'SET_CODE'
                                   ,right_on = 'code'
                                   ,how      = 'left')

# Dropping duplicate and redundant columns 
df__set_decks = df__set_decks.drop(columns = ['SET_DECKS', 'code'])

# Renaming the new columns
df__set_decks = df__set_decks.rename(columns__rename_set_decks
                                    ,axis = 1)

# Replacing the empty lists with NaN
df__set_decks = df__set_decks.map(lambda x: np.nan if isinstance(x, list) and len(x) == 0 else x)


#### Set Decks Info

In [ ]:
# Copying the decks source table and keeping key columns
df__set_decks_info = df__set_decks[columns__set_deck_info].copy()

# Extracting commanders into separate columns
df__set_decks_info = df__set_decks_info

In [ ]:
# Only keep rows where COMMANDER is not null
df_commanders = df__set_decks_info.loc[~df__set_decks['COMMANDER'].isna(), 'COMMANDER']

In [165]:
df_commanders.head()

735    [{'count': 1, 'uuid': '841f253f-e054-5d12-af3a...
736    [{'count': 1, 'uuid': '87e29342-ea21-5286-af30...
793    [{'count': 1, 'uuid': 'd3d4b8d4-4b2c-576c-ad36...
794    [{'count': 1, 'uuid': '127a011b-d742-5475-b0b0...
795    [{'count': 1, 'uuid': '64dfcece-1b6e-54f4-b109...
Name: COMMANDER, dtype: object

In [ ]:
df__set_decks_info[~df__set_decks_info['COMMANDER'].isna()].head(10)

,SET_CODE,SET_NAME,DECK_NAME,RELEASE_DATE,DECK_TYPE,COMMANDER
7147,TD0,Magic Online Theme Decks,Deathdancer Xira,2009-08-26,Commander Deck,"[{'count': 1, 'uuid': '841f253f-e054-5d12-af3a..."
7148,TD0,Magic Online Theme Decks,Enchantress Rubinia,2009-08-26,Commander Deck,"[{'count': 1, 'uuid': '87e29342-ea21-5286-af30..."
7151,TD0,Magic Online Theme Decks,Deathdancer Xira,2009-08-26,Commander Deck,"[{'count': 1, 'uuid': '841f253f-e054-5d12-af3a..."
7152,TD0,Magic Online Theme Decks,Enchantress Rubinia,2009-08-26,Commander Deck,"[{'count': 1, 'uuid': '87e29342-ea21-5286-af30..."
7155,TD0,Magic Online Theme Decks,Deathdancer Xira,2009-08-26,Commander Deck,"[{'count': 1, 'uuid': '841f253f-e054-5d12-af3a..."
7156,TD0,Magic Online Theme Decks,Enchantress Rubinia,2009-08-26,Commander Deck,"[{'count': 1, 'uuid': '87e29342-ea21-5286-af30..."
7159,TD0,Magic Online Theme Decks,Deathdancer Xira,2009-08-26,Commander Deck,"[{'count': 1, 'uuid': '841f253f-e054-5d12-af3a..."
7160,TD0,Magic Online Theme Decks,Enchantress Rubinia,2009-08-26,Commander Deck,"[{'count': 1, 'uuid': '87e29342-ea21-5286-af30..."
7783,CMD,Commander 2011,Counterpunch,2011-06-17,Commander Deck,"[{'count': 1, 'uuid': 'd3d4b8d4-4b2c-576c-ad36..."
7784,CMD,Commander 2011,Devour for Power,2011-06-17,Commander Deck,"[{'count': 1, 'uuid': '127a011b-d742-5475-b0b0..."


In [ ]:
# Only keep rows where COMMANDER is not null
df_commanders = df__set_decks.loc[~df__set_decks['COMMANDER'].isna(), 'COMMANDER']

# Expand the list of dictionaries into a DataFrame of UUIDs
# Each row will have one or two UUIDs (partner commanders)
commander_uuids = pd.DataFrame(
    df_commanders.apply(lambda x: [c['uuid'] for c in x] + [None]*(2-len(x))).tolist(),
    columns=['COMMANDER_1', 'COMMANDER_2'],
    index=df_commanders.index
)

# Merge back into the main dataframe
df__set_decks[['COMMANDER_1', 'COMMANDER_2']] = commander_uuids


In [20]:
# Only keep rows where COMMANDER is not null
df_commanders = df__set_decks.loc[~df__set_decks['COMMANDER'].isna(), 'COMMANDER']

# Expand the list of dictionaries into a DataFrame of UUIDs
# Each row will have one or two UUIDs (partner commanders)
commander_uuids = pd.DataFrame(
    df_commanders.apply(lambda x: [c['uuid'] for c in x] + [None]*(2-len(x))).tolist(),
    columns=['COMMANDER_1', 'COMMANDER_2'],
    index=df_commanders.index
)

# Merge back into the main dataframe
df__set_decks[['COMMANDER_1', 'COMMANDER_2']] = commander_uuids


In [21]:
df__set_decks = df__set_decks.drop(columns = 'COMMANDER')

In [22]:
df__set_decks = df__set_decks[['SET_CODE', 'SET_NAME', 'DECK_COUNT', 'DECK_NAME', 'RELEASE_DATE', 'DECK_TYPE',
                     'PARTNER_FLAG', 'COMMANDER_1', 'COMMANDER_2', 'DISPLAY_COMMANDER', 'DECK_CARDS', 
                     'SIDE_BOARD_CARDS', 'PLANES', 'SCHEMES', 'SEALED_PRODUCT_IDS']]

KeyError: "['PARTNER_FLAG'] not in index"

In [ ]:
df__set_decks.head()

,SET_CODE,SET_NAME,DECK_COUNT,DECK_NAME,RELEASE_DATE,DECK_TYPE,PARTNER_FLAG,COMMANDER_1,COMMANDER_2,DISPLAY_COMMANDER,DECK_CARDS,SIDE_BOARD_CARDS,PLANES,SCHEMES,SEALED_PRODUCT_IDS
0,LEA,Limited Edition Alpha,0,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,LEB,Limited Edition Beta,0,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2ED,Unlimited Edition,0,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,CED,Collectors' Edition,1,Collectors' Edition,1993-12-10,Box Set,False,NaN,NaN,NaN,"[{'count': 1, 'uuid': 'c423398e-f1d7-571c-8c05...",NaN,NaN,NaN,NaN
4,CEI,Intl. Collectors' Edition,1,Intl. Collectors' Edition,1993-12-10,Box Set,False,NaN,NaN,NaN,"[{'count': 1, 'uuid': '3648c747-d36e-5bd6-809d...",NaN,NaN,NaN,NaN


In [ ]:
df__set_decks[~df__set_decks['DISPLAY_COMMANDER'].isna()]['DISPLAY_COMMANDER'].iloc[0]

[{'count': 1, 'isFoil': True, 'uuid': '8e15d9b4-a7ce-5344-86b6-0c1fe118419d'},
 {'count': 1, 'isFoil': True, 'uuid': '84925c07-94df-5d6c-8411-fbc3eee337ef'},
 {'count': 1, 'isFoil': True, 'uuid': '36965d8a-cd92-5f76-8fb8-8055d64ef2a8'}]

In [ ]:
df__set_decks[~df__set_decks['SCHEMES'].isna()]['SCHEMES'].apply(len).unique()

array([15, 20, 10])

In [ ]:
columns__set_deck_list = ['SET_CODE'
                         ,'SET_NAME'
                         ,'DECK_COUNT'
                         ,'DECK_NAME'
                         ,'RELEASE_DATE'
                         ,'DECK_TYPE'
                         ,'PARTNER_FLAG'
                         ,'COMMANDER_1'
                         ,'COMMANDER_2']

df__set_deck_list = df__set_decks[columns__set_deck_list].copy()

In [ ]:
df__set_deck_list.head()

,SET_CODE,SET_NAME,DECK_COUNT,DECK_NAME,RELEASE_DATE,DECK_TYPE,PARTNER_FLAG,COMMANDER_1,COMMANDER_2
0,LEA,Limited Edition Alpha,0,NaN,NaN,NaN,False,NaN,NaN
1,LEB,Limited Edition Beta,0,NaN,NaN,NaN,False,NaN,NaN
2,2ED,Unlimited Edition,0,NaN,NaN,NaN,False,NaN,NaN
3,CED,Collectors' Edition,1,Collectors' Edition,1993-12-10,Box Set,False,NaN,NaN
4,CEI,Intl. Collectors' Edition,1,Intl. Collectors' Edition,1993-12-10,Box Set,False,NaN,NaN


In [ ]:
columns__set_deck_display_commanders = ['SET_CODE'
                                       ,'SET_NAME'
                                       ,'DECK_NAME'
                                       ,'DISPLAY_COMMANDER']

df__set_deck_display_commanders = df__set_decks[columns__set_deck_display_commanders].copy()

In [ ]:
df__set_deck_display_commanders['DISPLAY_COMMANDER'] = df__set_deck_display_commanders['DISPLAY_COMMANDER'].apply(lambda x: x if isinstance(x, list) else [])
df__set_deck_display_commanders = df__set_deck_display_commanders.explode('DISPLAY_COMMANDER', ignore_index=True)
df = pd.json_normalize(df__set_deck_display_commanders['DISPLAY_COMMANDER']).dropna()
df_final = pd.concat([df__set_deck_display_commanders, df], axis=1)

In [ ]:
scrollable_df(df_final[~df_final['DISPLAY_COMMANDER'].isna()])

,SET_CODE,SET_NAME,DECK_NAME,DISPLAY_COMMANDER,count,isFoil,uuid
793,CMD,Commander 2011,Counterpunch,"{'count': 1, 'isFoil': True, 'uuid': '8e15d9b4-a7ce-5344-86b6-0c1fe118419d'}",1.0,True,8e15d9b4-a7ce-5344-86b6-0c1fe118419d
794,CMD,Commander 2011,Counterpunch,"{'count': 1, 'isFoil': True, 'uuid': '84925c07-94df-5d6c-8411-fbc3eee337ef'}",1.0,True,84925c07-94df-5d6c-8411-fbc3eee337ef
795,CMD,Commander 2011,Counterpunch,"{'count': 1, 'isFoil': True, 'uuid': '36965d8a-cd92-5f76-8fb8-8055d64ef2a8'}",1.0,True,36965d8a-cd92-5f76-8fb8-8055d64ef2a8
796,CMD,Commander 2011,Devour for Power,"{'count': 1, 'isFoil': True, 'uuid': '3e456a02-3582-58b8-9591-3539b808f0bb'}",1.0,True,3e456a02-3582-58b8-9591-3539b808f0bb
797,CMD,Commander 2011,Devour for Power,"{'count': 1, 'isFoil': True, 'uuid': '62660eb2-98b5-507b-8ccd-9b9beba28589'}",1.0,True,62660eb2-98b5-507b-8ccd-9b9beba28589
798,CMD,Commander 2011,Devour for Power,"{'count': 1, 'isFoil': True, 'uuid': '8a26caad-c22d-5cf9-a919-7fee706ee626'}",1.0,True,8a26caad-c22d-5cf9-a919-7fee706ee626
799,CMD,Commander 2011,Heavenly Inferno,"{'count': 1, 'isFoil': True, 'uuid': 'deb78433-f189-5cfc-888d-34e70e990cd2'}",1.0,True,deb78433-f189-5cfc-888d-34e70e990cd2
800,CMD,Commander 2011,Heavenly Inferno,"{'count': 1, 'isFoil': True, 'uuid': '5662f43a-998e-55cc-9540-e36d0ff958eb'}",1.0,True,5662f43a-998e-55cc-9540-e36d0ff958eb
801,CMD,Commander 2011,Heavenly Inferno,"{'count': 1, 'isFoil': True, 'uuid': 'a0af6d22-1131-55ed-942f-4ee3abdc97ab'}",1.0,True,a0af6d22-1131-55ed-942f-4ee3abdc97ab
802,CMD,Commander 2011,Mirror Mastery,"{'count': 1, 'isFoil': True, 'uuid': 'adddb03e-27ab-532e-9e4e-f5bc860a8bfc'}",1.0,True,adddb03e-27ab-532e-9e4e-f5bc860a8bfc


In [ ]:
scrollable_df(df__set_deck_display_commanders[~df__set_deck_display_commanders['DISPLAY_COMMANDER'].isna()])

KeyError: 'DISPLAY_COMMANDER'

## Output

In [ ]:
'''
recency_check_upload(schema_name = "raw_data"
                    ,table_name  = "data_recency"
                    ,dataframe   = df__data_recency)
'''

In [ ]:
'''
# Uploading the keywords dataframe to postgresql
df__keywords.to_sql(name      = 'keywords'
                   ,con       = engine
                   ,schema    = 'raw_data'
                   ,if_exists = 'replace'
                   ,index     = False)
'''

212

## Checks

In [ ]:
'''
# Check the json file date and version
query = """
        SELECT *
        FROM raw_data.data_recency
        WHERE json_type = 'keyword'
        """
pd.read_sql_query(query, con=engine)
'''

,json_type,latest_date,latest_version
0,keyword,2025-08-19,5.2.2+20250819


In [ ]:
'''
# Check the dataframe top 10 values
query = """
        SELECT *
        FROM raw_data.keywords
        LIMIT 10
        """
pd.read_sql_query(query, con=engine)
'''

,abilities,keywords,actions
0,Adamant,Absorb,Abandon
1,Addendum,Affinity,Activate
2,Alliance,Afflict,Adapt
3,Battalion,Afterlife,Amass
4,Bloodrush,Aftermath,Assemble
5,Celebration,Amplify,Attach
6,Channel,Annihilator,Behold
7,Chroma,Ascend,Bolster
8,Cohort,Assist,Cast
9,Constellation,Augment,Clash
